In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import glob

In [2]:
import sys
print(sys.version)

3.7.9 (default, Aug 31 2020, 17:10:11) [MSC v.1916 64 bit (AMD64)]


In [3]:
# MAKE SURE datapath IS CORRECT
datapath = "E:\\WashU\\Neurotech_Club\\Data\\OneDrive-2020-11-28-2\\"
print(len(glob.glob(datapath+"*.csv")))
subjects = ["bradley", "johnathan", "vishal", "parker"] # there are no control files for bradley
count = 0

df_task = []
df_control = []

for subj in subjects:
    subj_files = glob.glob(datapath+"{}*.csv".format(subj))
    for f in subj_files:
        count+=1
        
        task_type = f.split("\\")[-1].split("_")[1] # nback or control
        
        print(task_type)
        temp = pd.read_csv(f, skiprows=1)

        if task_type == "nback":
            if len(df_task):
                df_task = pd.concat([df_task, temp], axis=0)
            else:
                df_task = temp
        elif task_type == "control":
            if len(df_control):
                df_control = pd.concat([df_control, temp], axis=0)
            else:
                df_control = temp
        else:
            None
            # TODO: warning

31
nback
nback
nback
nback
nback
control
control
nback
nback
nback
nback
nback
control
nback
nback
nback
nback
nback
control 
control
control
control
control
control
nback
nback
nback
nback
nback
nback
nback


In [4]:
print(df_task.shape)
df_task.head()

(173313, 151)


,Timestamp,EEG.Counter,EEG.Interpolated,EEG.AF3,EEG.F7,EEG.F3,EEG.FC5,EEG.T7,EEG.P7,EEG.O1,...,POW.F8.Theta,POW.F8.Alpha,POW.F8.BetaL,POW.F8.BetaH,POW.F8.Gamma,POW.AF4.Theta,POW.AF4.Alpha,POW.AF4.BetaL,POW.AF4.BetaH,POW.AF4.Gamma
0,1.606347e+09,19.0,0.0,4199.102539,4148.205078,4215.512695,4168.974121,4200.128418,4191.282227,4214.871582,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.606347e+09,20.0,0.0,4202.948730,4146.538574,4216.922852,4177.436035,4202.179688,4189.358887,4216.794922,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.606347e+09,21.0,0.0,4207.307617,4149.230957,4216.410156,4186.538574,4201.922852,4190.641113,4218.589844,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.606347e+09,22.0,0.0,4197.051270,4158.077148,4218.589844,4186.666504,4201.666504,4189.871582,4215.641113,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.606347e+09,23.0,0.0,4201.922852,4147.820313,4217.436035,4177.436035,4191.538574,4185.256348,4216.025879,...,8.043087,1.2325,1.862697,0.748757,0.440942,3.648506,1.338751,1.015494,0.474398,0.394639


In [5]:
print(df_control.shape)
df_control.head()

(189884, 151)


,Timestamp,EEG.Counter,EEG.Interpolated,EEG.AF3,EEG.F7,EEG.F3,EEG.FC5,EEG.T7,EEG.P7,EEG.O1,...,POW.F8.Theta,POW.F8.Alpha,POW.F8.BetaL,POW.F8.BetaH,POW.F8.Gamma,POW.AF4.Theta,POW.AF4.Alpha,POW.AF4.BetaL,POW.AF4.BetaH,POW.AF4.Gamma
0,1.606531e+09,60.0,0.0,4194.358887,4161.666504,4217.436035,4215.641113,4192.563965,4151.922852,4173.846191,...,1.948448,0.439749,1.209929,0.325765,0.149179,2.636014,0.63122,1.065295,0.638719,0.315357
1,1.606531e+09,61.0,0.0,4197.948730,4162.820313,4220.256348,4217.051270,4191.153809,4151.538574,4180.512695,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.606531e+09,62.0,0.0,4187.307617,4157.179688,4216.025879,4218.205078,4188.077148,4159.871582,4189.102539,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.606531e+09,63.0,0.0,4185.641113,4157.692383,4216.025879,4218.205078,4191.025879,4167.179688,4184.102539,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.606531e+09,64.0,0.0,4192.820313,4160.128418,4217.436035,4218.205078,4196.153809,4165.769043,4175.128418,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# df_task = pd.read_csv("Data/Parker/parker new nback 1_27.11.20_18.01.15.md.bp.csv", skiprows=1)
# df_control = pd.read_csv("Data/Parker/parker control 2_25.11.20_19.19.36.md.pm.bp.csv", skiprows=1)

df_task.dropna(subset=["POW.AF4.Alpha"], inplace=True)
df_control.dropna(subset=["POW.AF4.Alpha"], inplace=True)

# df_task.head()

df_task = df_task.filter(regex="Timestamp|EEG.AF|EEG.F|BetaL")
df_task = df_task.filter(regex="Timestamp|EEG.AF|EEG.F|POW.F|POW.AF")
# df_task['Timestamp'] = df_task['Timestamp']-df_task['Timestamp'].iloc[0]
df_task.set_index("Timestamp", inplace=True)
df_task["Label"] = 1

df_control = df_control.filter(regex="Timestamp|EEG.AF|EEG.F|BetaL")
df_control = df_control.filter(regex="Timestamp|EEG.AF|EEG.F|POW.F|POW.AF")
# df_control['Timestamp'] = df_control['Timestamp']-df_control['Timestamp'].iloc[0]
df_control.set_index("Timestamp", inplace=True)
df_control["Label"] = 0

In [7]:
# shuffle task and no task data
df_all = pd.concat([df_task, df_control])
df_all = df_all.sample(frac=1.0)

# normalize data
df_all_data = df_all.iloc[:, :-1]
df_all_labels = df_all.iloc[:,-1:]
df_all_data = (df_all_data - df_all_data.mean()) / df_all_data.std()

# put labels back on normalized data
df_all = pd.concat([df_all_data, df_all_labels], axis=1)

# Split into train/val sets
X_train = df_all.iloc[:int(0.7*len(df_all)), :-1].to_numpy()
Y_train = df_all.iloc[:int(0.7*len(df_all)), -1].to_numpy()

X_val = df_all.iloc[int(0.7*len(df_all)):, :-1].to_numpy()
Y_val = df_all.iloc[int(0.7*len(df_all)):, -1].to_numpy()

print(X_train.shape)
print(Y_train.shape)

print(X_val.shape)
print(Y_val.shape)

df_all.head()

(15890, 16)
(15890,)
(6811, 16)
(6811,)


,EEG.AF3,EEG.F7,EEG.F3,EEG.FC5,EEG.FC6,EEG.F4,EEG.F8,EEG.AF4,POW.AF3.BetaL,POW.F7.BetaL,POW.F3.BetaL,POW.FC5.BetaL,POW.FC6.BetaL,POW.F4.BetaL,POW.F8.BetaL,POW.AF4.BetaL,Label
Timestamp,,,,,,,,,,,,,,,,,
1.606528e+09,-0.086118,0.056220,-0.258752,-0.007933,-0.040323,-0.378387,-0.177679,-0.181163,-0.013579,-0.089123,-0.098306,-0.080276,-0.106069,-0.051514,-0.103620,-0.097977,0
1.606591e+09,-0.258997,0.167503,-0.265299,0.115232,0.003016,-0.030791,0.024179,0.091997,-0.081475,-0.085826,-0.099705,-0.068241,-0.072764,-0.066362,-0.090572,-0.076191,1
1.606599e+09,-0.347711,-0.286617,-0.429037,-0.228258,-0.184787,-0.721089,-0.255043,-0.765640,0.102073,-0.024406,0.080590,0.050698,0.071056,0.289426,0.055125,0.108815,0
1.606599e+09,0.946599,0.269791,0.484572,0.120704,0.102407,0.985083,0.272276,1.166554,-0.029357,-0.082985,-0.097583,-0.061552,-0.094458,-0.025884,-0.104152,-0.054944,0
1.606530e+09,0.089039,-0.120259,0.202954,0.012592,-0.012586,0.113640,-0.007833,-0.036547,-0.074897,-0.065971,-0.092887,-0.058001,-0.079006,-0.090689,-0.069523,-0.078822,1


In [8]:
def get_model(input_shape):
    x_input = Input(input_shape)
    x = Dense(32, activation="relu")(x_input)
    x = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=x_input, outputs=x)
    return model

In [9]:
model = get_model(X_train.shape[1]) # create an instance of the model
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 16)]              0         
_________________________________________________________________
dense (Dense)                (None, 32)                544       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 577
Trainable params: 577
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.compile(optimizer="adam", loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), metrics=[tf.keras.metrics.BinaryAccuracy(threshold=0.5)])

In [ ]:
history = model.fit(x=X_train,
                    y=Y_train,
                    validation_data=(X_val, Y_val),
                    batch_size=32,
                    verbose=1, # set to 0 for no epoch updates; 1 for updates
                    epochs=100)

Train on 15890 samples, validate on 6811 samples
Epoch 1/100


In [ ]:
loss = history.history["loss"]
val_loss = history.history["val_loss"]
binary_acc = history.history["binary_accuracy"]
val_binary_acc = history.history["val_binary_accuracy"]

fig, (ax1, ax2) = plt.subplots(1,2, figsize=(16, 5))

ax1.plot(loss)
ax1.plot(val_loss)
ax1.legend(["loss","val_loss"]);

ax2.plot(binary_acc)
ax2.plot(val_binary_acc)
ax2.legend(["binary_acc", "val_binary_acc"]);